### Graded Lab 3

Hello ! Welcome to Graded Lab of Module 3.

Working on the same analysis task.

* Here we will work on correlations between variables , testing few of the hypothesis business wants us to test.

In case if you are not able to recollect the problem description and data description then mentioning it below.

Lets look at the problem statement,

*Client: ABC Retail, Incorporated, rest-of-the-world division* 

***Project name: Online retail sales analysis*** 

An online retailer, ABC, Inc., operates in nearly 100 countries worldwide, selling furniture, office supplies and technology products to customers in three segments: consumer, corporate and home office. ABC, Inc. is a US-based company, and it has two major divisions: US and rest of the world. We are working with the rest of the world division of the company. 

They have provided us with online sales transaction data from 2011 to 2014.

We are given 3 datasets:-

1. Data on each sale; 51290 records; all data in US dollars
It contains fields like
**order_id** (identifier) ,order_date ,ship_date ,ship_mode ,**customer_id**(identifier) ,product_id ,category ,sub_category ,product_name ,sales ,quantity ,discount ,profit ,shipping_cost ,order_priority ,**vendor_code** (identifier) 


2. Data on the customers; 1590 records 
It contains fields like
**customer_id** (identifier) ,customer_name ,city ,state ,country ,postal_code ,segment ,market ,region 

3. Data on vendors who supply the retailer; 65 records 
It contains fields like
vendor ,**vendor_code** (identifier) 

We need to analyze the data and need to provide answers to different questions asked by company officials.

In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from itertools import combinations
from scipy import stats
from statsmodels.stats import weightstats as ztests


### Reading sales data
sales = pd.read_csv('sales_data.csv')

### Reading customer data
cust = pd.read_csv(r'customers.csv',encoding='iso-8859-1')

### Reading vendor data
vend = pd.read_csv(r'vendors.csv')

sales.head()

In [ ]:
pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.4f}'.format)

In [ ]:
sales.shape, cust.shape, vend.shape

In [ ]:
cust.head()

In [ ]:
vend.head()

In order to solve the next questions , we need to combine all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. So here we have written a data processing function.
There are 2 tasks which are to be performed.
1. Merge/ Join all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. (Understand which should be the joining key , type of join , refer .merge() function of pandas)
2. Convert 'order_date' into a datetime column.
**Return output as a dataframe**

In [ ]:
#### data_merging & order_date processing , data1 will be sales , data2 will be customer dataset & data3 will be vendor dataset.

def data_process(data1,data2,data3):
    df = data1.merge(right=data3, how="left", on="vendor_code")
    df2 = df.merge(right=data2, how="left", on="customer_id")
    df2["order_date"] = pd.to_datetime(df2['order_date'], format='%d/%m/%Y')
    return df2

In [ ]:
sales= data_process(data1=sales.copy(),data2=cust.copy(),data3=vend.copy())

In [ ]:
sales.shape

In [ ]:
assert sales['order_date'].dtypes=='<M8[ns]' ,'Make sure that you have converted order_date into a datetime format correctly.'
assert sales.shape== (51290,26) ,'Checking size and shape of dataframe after merging is a very important check.'

### Q1. What is skewness,skewness type ,kurtosis, kurtosis type for profit column ? 
#### Return output in tuple format.Eg :- (-1, "Left-skewed", 2, "Platykurtic"). Make sure to round off answer to 2 decimals.

Note:-
1. Skewness type takes 3 values :- Right-skewed (Skewness >0) ,Left-skewed (Skewness <0), Symmetric (Skewness = 0)
2. Kurtosis type takes 3 values :- Platykurtic (kurtosis < 3) ,Leptokurtic (kurtosis > 3), Mesokurtic (kurtosis = 3)

In [ ]:
skewness = round(stats.skew(sales.profit), 2)

In [ ]:
skewness

In [ ]:
skewness_type = "Right-skewed" if skewness > 0 else "Left-skewed" if skewness < 0 else "Symmetric"
skewness_type

In [ ]:
kurtosis = round(stats.kurtosis(sales.profit), 2)
kurtosis

In [ ]:
kurtosis_type = "Leptokurtic" if kurtosis > 3 else "Platykurtic" if kurtosis < 3 else "Mesokurtic"
kurtosis_type

In [ ]:
result = (skewness, skewness_type, kurtosis, kurtosis_type)
result

*** ----------------------------------------------------------- ***

In [ ]:
def skew_kurt(data,column):  
    
    column_data = data[column]
    
    skewness = round(stats.skew(column_data), 2)
    skewness_type = "Right-skewed" if skewness > 0 else "Left-skewed" if skewness < 0 else "Symmetric"

    kurtosis = round(stats.kurtosis(column_data), 2)
    kurtosis_type = "Leptokurtic" if kurtosis > 3 else "Platykurtic" if kurtosis < 3 else "Mesokurtic"

    result = (skewness, skewness_type, kurtosis, kurtosis_type)
    
    return result

In [ ]:
assert type(skew_kurt(data=sales,column='profit'))==tuple, "Please provide output in a tuple format."
assert len(skew_kurt(data=sales,column='profit'))==4,"Please include skewness , skewness_type , kurtosis & kurtosis_type."
assert skew_kurt(data=sales,column='profit')[1] in ["Right-skewed","Left-skewed","Symmetric"],"Please include from the listed types only , also make sure that spelling is correct."
assert skew_kurt(data=sales,column='profit')[3] in ["Platykurtic","Leptokurtic","Mesokurtic"],"Please include from the listed types only , also make sure that spelling is correct."

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

### Q2.	What is correlation between different numerical variables ? ('sales', 'quantity', 'discount', 'profit', 'shipping_cost')
### Return a dataframe with 3 columns : Variable 1 , Variable 2 , Correlation. Make sure to sort dataframe with respect to highest absolute correlation & one variable pair occurs only once.

In [ ]:
sales.columns

In [ ]:
df = sales[['sales', 'quantity', 'discount', 'profit', 'shipping_cost']]
df.head()

In [ ]:
# Calculate the correlation matrix
correlation_matrix = df.corr()
correlation_matrix

In [ ]:
# Create a DataFrame to store the correlation values and their pairs
correlation_data = []

# Iterate through the upper triangle of the correlation matrix to avoid duplicate pairs
for i in range(len(df.columns)):
    for j in range(i + 1, len(df.columns)):
        variable_1 = df.columns[i]
        variable_2 = df.columns[j]
        corr = correlation_matrix.iloc[i, j]
        correlation_data.append([variable_1, variable_2, corr])

In [ ]:
correlation_data

In [ ]:
# Create the DataFrame with the correlation values
pair_df = pd.DataFrame(correlation_data, columns=['Variable 1', 'Variable 2', 'Correlation'])
pair_df

In [ ]:
# Sort the DataFrame by the absolute correlation values in descending order
pair_df = pair_df.reindex(pair_df['Correlation'].abs().sort_values(ascending=False).index)

# Reset the index of the resulting DataFrame
pair_df = pair_df.reset_index(drop=True)

In [ ]:
pair_df

In [ ]:
def corr_sales(data):
    data = data[['sales', 'quantity', 'discount', 'profit', 'shipping_cost']]
    
    correlation_matrix = data.corr()
    
    correlation_data = []
    
    for i in range(len(data.columns)):
        for j in range(i + 1, len(data.columns)):
            variable_1 = data.columns[i]
            variable_2 = data.columns[j]
            corr = correlation_matrix.iloc[i, j]
            correlation_data.append([variable_1, variable_2, corr])
    
    pair_df = pd.DataFrame(correlation_data, columns=['variable_1', 'variable_2', 'corr'])
    
    pair_df = pair_df.reindex(pair_df['corr'].abs().sort_values(ascending=False).index)
    
    pair_df = pair_df.reset_index(drop=True)
    
    return pair_df[['variable_1','variable_2','corr']]

In [ ]:
corr_sales(sales)

In [ ]:
assert type(corr_sales(data=sales))==pd.DataFrame, "Please provide output in dataframe format."
assert all(corr_sales(data=sales).columns==['variable_1', 'variable_2', 'corr']), "Make sure your output columns are named as 'variable_1', 'variable_2', 'corr'."
assert corr_sales(data=sales).shape[1]==3, "Make sure you are returning only 3 columns."
assert corr_sales(data=sales).shape[0]==10, "Make sure you are returning all possible combinations."

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

### Q3.	 Plot a box plot for sales in 'Furniture' category and tell on which side of box-plot you were able to see outliers.

### Return output from one of the following choices : 1. both_sides , 2. upper_whisker , 3. lower_whisker

In [ ]:
sales.category.unique()

In [ ]:
furniture_data = sales[sales['category'] == 'Furniture']
furniture_data.head()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=furniture_data, x="sales")
plt.show()

In [ ]:
def outlier_sales(data): 
    furniture_data = data[data['category'] == 'Furniture']
    sns.boxplot(data=furniture_data, x="sales")
    outliers_whiskers = "upper_whisker"
    return outliers_whiskers # outlier_whsikers is a variable in where you will store answer selected from this 3 options : 1. both_sides , 2. upper_whisker , 3. lower_whisker

In [ ]:
outlier_sales(sales)

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

### Q4.	What is probability that profit lies between 13 & 25.
Return output as float type ,rounded upto 4 decimals.

In [ ]:
sales.profit.values

In [ ]:
lower_bound = 13
upper_bound = 25

In [ ]:
profits_in_range = [profit for profit in sales.profit.values if lower_bound <= profit <= upper_bound]

In [ ]:
profits_in_range [:5]

In [ ]:
probability = len(profits_in_range) / len(sales.profit.values)

In [ ]:
probability = round(probability, 4)

In [ ]:
np.float64(probability)

In [ ]:
type(probability)

In [ ]:
def prob(data,lower_bound,upper_bound):
    
    data = data.profit.values
    profits_in_range = [profit for profit in data if lower_bound <= profit <= upper_bound]
    probability = len(profits_in_range) / len(data)
    probability = round(probability, 4)
    probability = np.float64(probability)
    return probability

In [ ]:
prob(sales, 13, 25)

In [ ]:
assert type(prob(data=sales,lower_bound=13,upper_bound=25))==np.float64, 'Make sure that you are returning output as float type , it should be rounded up to 4 decimals.'
assert (prob(data=sales,lower_bound=13,upper_bound=25)>=0) & (prob(data=sales,lower_bound=13,upper_bound=25)<=1), 'Remember probability always lies between 0 to 1.'

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

###  Q.5) Company's Sales team found an interesting insight from past study. Study said that on a 3-yr scale  avg quantities sold comes out to be 3. Now they want to test if the avergae quantities sold number has increased or not i.e they wish to understand if average quantities sold are greater than a previously known value i.e 3 or not , Our data is on  3 yrs horizon. So they have asked you to draw a sample of size 300 from sales data, analyze and report findings. Select alpha (level of significance) as 0.05.

**Return output as tuple consisting of  (statistics_value, p-value, decision). Make sure to round up statistics_value & p-value to 4 decimals. Your code will be tested on an input sample (sample_1).**


For eg :- (1.267,0.045,'Reject Null hypothesis')

Where ,stastics:- It is calculated statistic from appropriate test.

p-value :- probability value

Decision :- You have to select from the 2 choices 1.'Reject Null Hypothesis' , 2.'Do not Reject Null Hypothesis''

**Hint:**

We are trying to test a hypothesis whether average quantities data for sales data are greater than previously known value 3 or not ?
In order to solve this problem , Think on the lines below.

H0: Average quantities sold = previously_known_population_value (3)

H1: Average quantities sold > previously_known_population_value (3) 

We will use 1 sample z-test with alernative type as 'larger'  for solving this problem.

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell.

In [ ]:
sales.head()

In [ ]:
def one_sample_ztest(sample1,alpha,popn_value):
    ## sample1(quantity) will be a list of values with length 100.
    # Calculate the sample mean and standard error
    sample_mean = sum(sample1) / len(sample1)
    sample_std = (sum([(x - sample_mean) ** 2 for x in sample1]) / (len(sample1) - 1)) ** 0.5
    sample_size = len(sample1)
    
    # Calculate the Z-statistic
    z_statistic = (sample_mean - popn_value) / (sample_std / (sample_size ** 0.5))
    
    # Calculate the p-value
    p_value = 1 - stats.norm.cdf(z_statistic)
    
    # Make a decision based on the alpha level
    if p_value < alpha:
        decision = "Reject H0"
    else:
        decision = "Fail to reject H0"
    
    return round(z_statistic,4), round(p_value,4), decision  

In [ ]:
sales["quantity"].value_counts()

In [ ]:
sample1 = sales["quantity"].sample(n=300)

In [ ]:
alpha = 0.05

In [ ]:
popn_value = 3

In [ ]:
one_sample_ztest(sample1,alpha,popn_value)

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.6)  Sales team is interested in knowing if furniture & office supply products sell equally or is there any differece between their sells so they ask us to draw a sample of size 22 from furniture & sample of size 25 from Office Supplies from sales dataset and report findings. Select alpha (level of significance) as 0.05.

**Return output as tuple consisting of (statistics_value, p-value, decision). Make sure that stastics,p-value are rounded upto 4 digits.Your code will be tested on 2 input samples (sample_1,sample_2).**

For eg :- (1.267,0.045,'Reject Null hypothesis')

Where ,stastics:- It is calculated statistic from chosen test.

p-value :- probability value

Decision :- 'Reject Null Hypothesis' , ''Do not Reject Null Hypothesis''



In [ ]:
sales["category"].value_counts()

In [ ]:
furnsales = sales[sales["category"] == "Furniture"].sample(n=22)

In [ ]:
furnsales.head()

In [ ]:
sample1 = furnsales[["sales"]]

sample1

In [ ]:
officesales = sales[sales["category"] == "Office Supplies"].sample(n=25)

In [ ]:
officesales.head()

In [ ]:
sample2 = officesales[["sales"]]

sample2

In [ ]:
statistic, p_value = stats.ttest_ind(sample1, sample2, equal_var=False)

In [ ]:
statistic

In [ ]:
p_value

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell.

In [ ]:
def hypo_test(sample1,sample2,alpha):
    ## sample1(furniture sales) will be a list of values with length 22.
    ## sample2(office supplies sales) will be a list of values with length 25.
    # your code here
    
    
    
    # Perform a two-sample t-test
    statistic, p_value = stats.ttest_ind(sample1, sample2, equal_var=False)
    
    # Check if the p-value is less than alpha to make a decision
    if p_value < alpha:
        decision = "Reject null hypothesis"
    else:
        decision = "Do not Reject Null Hypothesis"
        
    # Round the statistic and p-value for output
    statistic = statistic[0]
    p_value = p_value[0]
    
    #statistic = round(statistic, 4)
    #p_value = round(p_value, 4)
        
    return round(statistic, 4), round(p_value, 4), decision

In [ ]:
sample_1 = sample1
sample_2 = sample2
alpha = 0.05

In [ ]:
hypo_test(sample_1,sample_2,alpha)

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell
### Hidden test 1

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell
### Hidden test 2

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell
### Hidden test 2

## Great Job ! ! !
Business team is happy with your work they wish to test couple of more things & require your help.

### Q.6)  Sales team is interested in knowing if profits in EMEA & Africa region are same or not. Test the hypothesis. Select alpha (level of significance) as 0.05.

**Return output as tuple consisting of (statistics_value,p-value,decision). Make sure that stastics,p-value are rounded upto 4 digits.Your code will be tested on 2 input samples (sample_1,sample_2).**


For eg :- (1.267,0.045,'Reject Null hypothesis')

Where ,statistics_value:- It is calculated statistic from chosen test.

p-value :- probability value

Decision :- 'Reject Null Hypothesis' , ''Do not Reject Null Hypothesis''



In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

In [ ]:
def hypo_test(sample1,sample2,alpha):
    ## sample_1 (EMEA region) will be a list of values with length 500.
    ## sample_2 (Africa region) will be a list of values with length 567.
    
    
    # Perform a two-sample t-test
    statistic, p_value = stats.ttest_ind(sample1, sample2, equal_var=False)
    
    # Check if the p-value is less than alpha to make a decision
    if p_value < alpha:
        decision = "Reject null hypothesis"
    else:
        decision = "Do not Reject Null Hypothesis"
        
    # Round the statistic and p-value for output
    statistic = statistic[0]
    p_value = p_value[0]
    
    return round(statistic,4),round(p_value,4),decision

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell
### Hidden test 1

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell
### Hidden test 2

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell
### Hidden test 3

# 15/100 points earned